In [5]:
### Run this to be able to access your drive. Google will ask you to input a verification code twice. Just do so

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [6]:
## Ensure your drive contains the folder ML Captcha at this bottom address. Run this to test if it really is there.

!mkdir -p drive
!google-drive-ocamlfuse drive
import os
os.chdir("/content/drive/ML Captcha")
!ls

dodyfolderdata.pickledody		     ML_model (temp).hdf5
dodyfolderlabels.pickledody		     ML_model_test.pb
drive					     ML Train.ipynb
events.out.tfevents.1531566458.ab523be53b33  Modified_Claptcha_Lib.py
ML_Data_Quicksand			     __pycache__
ML_model.hdf5				     Quicksand-Regular.otf
ML_model_labels.dat			     Test_ML_Model.ipynb
ML_model_labels (temp).dat		     Visualise_Model.ipynb


In [79]:
from keras import backend as K
from keras.models import load_model
from keras import backend as K
import cv2
import tensorflow as tf

FOLDER = "/content/drive/ML Captcha/"

image_file = FOLDER + "ML_Data_Quicksand/Q/1.png"

# Load the image and convert it to grayscale
image = cv2.imread(image_file)
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Add 2 channel dimensions to the image to make Keras happy
image = np.expand_dims(image, axis=2)
image = np.expand_dims(image, axis=0)

print (image.shape)

# Load the relevant files
MODEL_FILENAME = FOLDER + "ML_model.hdf5"

# Load model
model = load_model(MODEL_FILENAME)

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers])

print (layer_dict)
print (tf.convert_to_tensor(image))

layer_name = 'conv2d_5'
filter_index = 1  # can be any integer from 0 to 511, as there are 512 filters in that layer

# build a loss function that maximizes the activation
# of the nth filter of the layer considered
layer_output = layer_dict[layer_name].output
print (layer_output)
loss = K.mean(layer_output[:, :, :, 1])
print (loss)

grads = K.gradients(loss, 1)[0]
print (grads)

(1, 20, 20, 1)
{'conv2d_5': <keras.layers.convolutional.Conv2D object at 0x7ff46edcd358>, 'max_pooling2d_5': <keras.layers.pooling.MaxPooling2D object at 0x7ff46fd34208>, 'conv2d_6': <keras.layers.convolutional.Conv2D object at 0x7ff470ea2160>, 'max_pooling2d_6': <keras.layers.pooling.MaxPooling2D object at 0x7ff4712b76d8>, 'flatten_3': <keras.layers.core.Flatten object at 0x7ff470ea2240>, 'dense_5': <keras.layers.core.Dense object at 0x7ff46ecd8320>, 'dense_6': <keras.layers.core.Dense object at 0x7ff46ed262b0>}
Tensor("Const_47:0", shape=(1, 20, 20, 1), dtype=uint8)
Tensor("conv2d_5_50/Relu:0", shape=(?, 20, 20, 20), dtype=float32)
Tensor("Mean_40:0", shape=(), dtype=float32)
None


In [65]:
from keras import applications

image_file = FOLDER + "ML_Data_Quicksand/Q/1.png"
image = cv2.imread(image_file)

# build the VGG16 network
model = applications.VGG16(include_top=False,
                           weights='imagenet')

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers])

from keras import backend as K

layer_name = 'block5_conv3'
filter_index = 0  # can be any integer from 0 to 511, as there are 512 filters in that layer

# build a loss function that maximizes the activation
# of the nth filter of the layer considered
layer_output = layer_dict[layer_name].output
print (layer_output)
loss = K.mean(layer_output[:, :, :, filter_index])
print (loss)

# compute the gradient of the input picture wrt this loss
grads = K.gradients(loss, image)[0]
print (grads)

# normalization trick: we normalize the gradient
grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

# this function returns the loss and grads given the input picture
iterate = K.function([input_img], [loss, grads])

Tensor("block5_conv3_1/Relu:0", shape=(?, ?, ?, 512), dtype=float32)
Tensor("Mean_31:0", shape=(), dtype=float32)
None


ValueError: ignored

In [3]:
from keras.models import load_model

FOLDER = "/content/drive/ML Captcha/"

# Load the model file
MODEL_FILENAME = FOLDER + "ML_model_test.pb"

# Load model
model = load_model(MODEL_FILENAME)

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers])

print (layer_dict)

OSError: ignored

In [18]:
from keras.models import load_model

FOLDER = "/content/drive/ML Captcha/"

# Load the model file
MODEL_FILENAME = FOLDER + "ML_model.hdf5"

# Load model
model = load_model(MODEL_FILENAME)

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers])

print (layer_dict)

{'conv2d_5': <keras.layers.convolutional.Conv2D object at 0x7fe261abd5c0>, 'max_pooling2d_5': <keras.layers.pooling.MaxPooling2D object at 0x7fe261cfe4a8>, 'conv2d_6': <keras.layers.convolutional.Conv2D object at 0x7fe261cfe160>, 'max_pooling2d_6': <keras.layers.pooling.MaxPooling2D object at 0x7fe261cfe0f0>, 'flatten_3': <keras.layers.core.Flatten object at 0x7fe261ae7c50>, 'dense_5': <keras.layers.core.Dense object at 0x7fe261ae7898>, 'dense_6': <keras.layers.core.Dense object at 0x7fe261a8da58>}


In [44]:
# boilerplate code
from __future__ import print_function
import os
from io import BytesIO
import numpy as np
from functools import partial
import PIL.Image
from IPython.display import clear_output, Image, display, HTML

import tensorflow as tf

FOLDER = "/content/drive/ML Captcha/"

model_fn = FOLDER + "ML_model_test.pb"

graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)
with tf.gfile.FastGFile(model_fn, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)

#t_input = tf.placeholder(np.float32, name='input') # define the input tensor
#print(t_input)
#imagenet_mean = 117.0
#t_preprocessed = tf.expand_dims(t_input-imagenet_mean,0)
#print (t_preprocessed)
#tf.import_graph_def(graph_def, {'input':t_preprocessed})

layers = [op.name for op in graph.get_operations() if op.type=='Conv2D' and 'import/' in op.name]
feature_nums = [int(graph.get_tensor_by_name(name+':0').get_shape()[-1]) for name in layers]

print('Number of layers', len(layers))
print('Total number of feature channels:', sum(feature_nums))


# Helper functions for TF Graph visualization

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def
  
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

# Visualizing the network graph. Be sure expand the "mixed" nodes to see their 
# internal structure. We are going to visualize "Conv2D" nodes.
tmp_def = rename_nodes(graph_def, lambda s:"/".join(s.split('_',0)))
show_graph(graph_def)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Number of layers 2
Total number of feature channels: 70


In [43]:
import tensorflow as tf

def printTensors(pb_file):

    # read pb into graph_def
    with tf.gfile.GFile(pb_file, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # import graph_def
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)

    # print operations
    for op in graph.get_operations():
        print(op.name)


printTensors(model_fn)

import/conv2d_1/kernel
import/conv2d_1/bias
import/conv2d_2/kernel
import/conv2d_2/bias
import/dense_1/kernel
import/dense_1/bias
import/dense_2/kernel
import/dense_2/bias
import/Adam/iterations
import/Adam/lr
import/Adam/beta_1
import/Adam/beta_2
import/Adam/decay
import/training/Adam/Variable
import/training/Adam/Variable_1
import/training/Adam/Variable_2
import/training/Adam/Variable_3
import/training/Adam/Variable_4
import/training/Adam/Variable_5
import/training/Adam/Variable_6
import/training/Adam/Variable_7
import/training/Adam/Variable_8
import/training/Adam/Variable_9
import/training/Adam/Variable_10
import/training/Adam/Variable_11
import/training/Adam/Variable_12
import/training/Adam/Variable_13
import/training/Adam/Variable_14
import/training/Adam/Variable_15
import/training/Adam/Variable_16
import/training/Adam/Variable_17
import/training/Adam/Variable_18
import/training/Adam/Variable_19
import/training/Adam/Variable_20
import/training/Adam/Variable_21
import/training/Adam